# Import modules

In [ ]:
import matplotlib.pyplot as plt # for plotting
import numpy as np # for numpy
import math
from math import pi, sin, cos # for math
DEGREES_TO_RADIANS = pi / 180
from math import isnan
from ect import ECT, EmbeddedGraph # for creating ECT
from sklearn.manifold import MDS # for MDS
import pandas as pd
import seaborn as sns

#---------------------------
# The PHATE packages we'll use
#---------------------------
import phate # for using PHATE # pip install --user phate
import scprep # for using PHATE

# Define functions

In [ ]:
def superformula(phi, m, n1, n2, n3, a, b):
    r = (np.abs((np.cos((m*phi)/4))/a)**n2 + np.abs((np.sin((m*phi)/4))/b)**n3)**(-1/n1)
    return r

def pol2cart(rho, phi):
    x = rho * np.cos(phi)
    y = rho * np.sin(phi)
    return(x, y)
    
def cart2pol(x, y):
    rho = np.sqrt(x**2 + y**2)
    phi = np.arctan2(y, x)
    return(rho, phi)

# Superformula

To generate shapes *de novo*, we will be using the superformula:

Johan Gielis. (2003). [**A generic geometric transformation that unifies a wide range of natural and abstract shapes.**](https://bsapubs.onlinelibrary.wiley.com/doi/10.3732/ajb.90.3.333). *Am J Bot.* 90(3):333-8

An interesting article about the parameters to choose for the super formula:
[https://paulbourke.net/geometry/supershape/](https://paulbourke.net/geometry/supershape/)

Below we loop over parameters and generate some shapes! We both visulize and save the shapes as `x` and `y` coordinate values for further analysis.

In [ ]:
angs = 360 # number of angles to sample

phi_vals = np.linspace(0,2*math.pi,angs) # we are in radians, so we generate some angles to use

m_vals = [2,3,4,5,6,7,8] # m values to loop over
n1_vals = [5,30,70] # n1 vals
n2_vals = [15,50,90] # n2 vals
n3_vals = [25,70,110] # n3 vals
a_vals = [1,2,4] # a vals
b_vals = [1,5,7] # b vals

# lists to store associated values of shapes
m_list = []
n1_list = []
n2_list = []
n3_list = []
a_list = []
b_list = []

# colors, by variable "m"
cols = ["#ff0000", "#fc4444", "#fc6404", "#fcd444", "#8cc43c", "#029658", "#1abc9c", "#5bc0de", "#6454ac", "#fc8c84"]

# to store x and y vals for analysis
coord_arr = np.zeros((len(m_vals)*len(n1_vals)*len(n2_vals)*len(n3_vals)*len(a_vals)*len(b_vals), 
                     angs, 
                     2))

plot_num = 1 # a counter for the subplot

plt.figure(figsize=(50,30))

for m in m_vals:
    col = cols[m-2] # change the color when the m value changes
    for n1 in n1_vals:
        for n2 in n2_vals:
            for n3 in n3_vals:
                for a in a_vals:
                    for b in b_vals:
                
                        r_vals = superformula(phi_vals, m,n1,n2,n3,a,b)
                        
                        xvals, yvals = pol2cart(r_vals, phi_vals) # convert to x and y coordinate values
                        arr = np.array((xvals, yvals)).T # place x and y vals into array
                        coord_arr[plot_num-1,:,:] = arr # store shape to array
        
                        plt.subplot(63,27,plot_num)
                        plt.fill(xvals, yvals, c=col)
                        plt.gca().set_aspect("equal")
                        plt.axis("off")
                        
                        plot_num += 1
                        if plot_num%100==0:
                            print(plot_num)

                        # store shape parameter values to list
                        m_list.append(m)
                        n1_list.append(n1)
                        n2_list.append(n2)
                        n3_list.append(n3)
                        a_list.append(a)
                        b_list.append(b)
                        



plt.tight_layout()
                

In [ ]:
# create df of parameter values
labels_df = pd.DataFrame({
    "m":m_list,
    "n1":n1_list,
    "n2":n2_list,
    "n3":n3_list,
    "a":a_list,
    "b":b_list})

# convert to strings, because small number
labels_df = labels_df.astype(str)

# check df
labels_df.head()

# Calculate ECTs

In [ ]:
# get number of shapes
shape_num = len(m_vals)*len(n1_vals)*len(n2_vals)*len(n3_vals)*len(a_vals)*len(b_vals)

# specify number of directions and threshold
num_dirs = 32
num_thresh = 48

# to store ECTs
ect_arr = np.zeros((shape_num,
                    num_dirs,
                    num_thresh))

for i in range(shape_num):

    if i%100==0:
        print(i)

    shape = coord_arr[i,:,:] # retrieve the current shape from the shape array
    
    G = EmbeddedGraph() # initiate graph
    G.add_cycle(shape) # add the shape coordinates as a graph cycle
    G.set_PCA_coordinates( center_type='min_max', scale_radius=1) # normalize the shape coordinates
    
    # initialize ECT
    myect = ECT(num_dirs = num_dirs, num_thresh = num_thresh)

    #calculate ECT
    myect.calculateECT(G)

    # The matrix is passed as an output above but is also saved internally. Get the saved matrix
    M = myect.get_ECT()
    
    # save ECT
    ect_arr[i, :, :] = M

In [ ]:
plt.figure(figsize=(20,50))

counter = 1

for i in range(np.shape(ect_arr)[0]):
        
    curr_ect = ect_arr[i,:,:]
    plt.subplot(63,27,counter)
    plt.imshow(curr_ect.T)
    plt.axis("off")
    
    counter +=1
        
plt.tight_layout()

________

# PHATE (Potential of Heat-diffusion for Affinity-based Trajectory Embedding)

We will be using [PHATE](https://github.com/KrishnaswamyLab/PHATE) to visualize the manifold underlying pavement cell shapes. You can learn more about PHATE from the following reference:

Moon, van Dijk, Wang, Gigante et al. **[Visualizing Transitions and Structure for Biological Data Exploration](https://www.nature.com/articles/s41587-019-0336-3)**. 2019. *Nature Biotechnology*.

Before using PHATE, we will need to flatten the ect array

In [ ]:
np.shape(ect_arr)

In [ ]:
# flatten the cell ECT data to a 2D array by sample
flat_ect = np.reshape(ect_arr,
                             (np.shape(ect_arr)[0],
                              np.shape(ect_arr)[1]*np.shape(ect_arr)[2])
                            )

### PHATE in 2 dimensions

In [ ]:
# create the PHATE embedding and transform
phate_operator = phate.PHATE(n_components=2)
phate_super = phate_operator.fit_transform(flat_ect)

Add the PHATE axes to the dataframe for plotting

In [ ]:
labels_df["2Dphate1"] = phate_super[:,0]
labels_df["2Dphate2"] = phate_super[:,1]

Create a PHATE plot in 2D

In [ ]:
plt.figure(figsize=(4*4,6*4)) # set figure size
pt_size = 50 # set point size
pal = "inferno" # set palette

plt.subplot(3,2,1)
sns.scatterplot(data=labels_df, x="2Dphate1", y="2Dphate2", hue="m", legend=True, s=pt_size)
plt.title("PHATE1 and PHATE2\nby m")

plt.subplot(3,2,2)
sns.scatterplot(data=labels_df, x="2Dphate1", y="2Dphate2", hue="n1", legend=True, s=pt_size)
plt.title("PHATE1 and PHATE2\nby n1")

plt.subplot(3,2,3)
sns.scatterplot(data=labels_df, x="2Dphate1", y="2Dphate2", hue="n2", legend=True, s=pt_size)
plt.title("PHATE1 and PHATE2\nby n2")

plt.subplot(3,2,4)
sns.scatterplot(data=labels_df, x="2Dphate1", y="2Dphate2", hue="n3", legend=True, s=pt_size)
plt.title("PHATE1 and PHATE2\nby n3")

plt.subplot(3,2,5)
sns.scatterplot(data=labels_df, x="2Dphate1", y="2Dphate2", hue="a", legend=True, s=pt_size)
plt.title("PHATE1 and PHATE2\nby a")

plt.subplot(3,2,6)
sns.scatterplot(data=labels_df, x="2Dphate1", y="2Dphate2", hue="b", legend=True, s=pt_size)
plt.title("PHATE1 and PHATE2\nby b")

plt.tight_layout()

### PHATE in 3 dimensions

In [ ]:
# create the PHATE embedding and transform
phate_operator = phate.PHATE(n_components=3)
phate_super = phate_operator.fit_transform(flat_ect)

Add the PHATE axes to the dataframe for plotting

In [ ]:
labels_df["3Dphate1"] = phate_super[:,0]
labels_df["3Dphate2"] = phate_super[:,1]
labels_df["3Dphate3"] = phate_super[:,2]

Create a 3D animation by type

In [ ]:
sample_labels = labels_df["m"]

# for a static plot use "scatter3d" instead of "rotate_scatter3d"
# this saves to GIF and produces animation
scprep.plot.rotate_scatter3d(phate_super, c=sample_labels, figsize=(8*2,6*2), ticks=True, label_prefix="PHATE", filename="super_M.gif", s=30)


________

# Save files

### let us save the ECT array as a numpy `.npy` file

In [ ]:
np.save("superformula_ect_arr.npy", ect_arr)

### let us also save the associated pandas dataframe as `.csv`

In [ ]:
labels_df.to_csv("superformula_df.csv")

________